Github link:

In [ ]:
# Mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
PROJ_DIR = '/content/drive/MyDrive/thwisha/AIPI Assignment-3/SA2C_code/SA2C_code/Kaggle'   ## give your drive folder location
# change current directory after mounting
%cd $PROJ_DIR
! ls

/content/drive/.shortcut-targets-by-id/1mlkwkqJvfjZrMiy4ViJx-_vpAevMQXKU/AIPI Assignment-3/SA2C_code/SA2C_code/Kaggle
data		     preprocess_kaggle.py  report_SNQN.txt   SNQN_Features_new.py  split_data.py
DQN_NS.py	     __pycache__	   SA2C_new.py	     SNQN_Features.py	   test.py
NextItNetModules.py  replay_buffer.py	   SA2C.py	     SNQN_new.py	   utility.py
pop.py		     report_SA2C.txt	   SASRecModules.py  SNQN.py


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_category=pd.read_csv("/content/drive/MyDrive/AIPI Assignment-3/category_tree.csv")
df_events=pd.read_csv("/content/drive/MyDrive/AIPI Assignment-3/events.csv")
df_items1=pd.read_csv("/content/drive/MyDrive/AIPI Assignment-3/item_properties_part1.csv")
df_items2=pd.read_csv("/content/drive/MyDrive/AIPI Assignment-3/item_properties_part2.csv")

In [ ]:
print("Shape of category dataset:", df_category.shape)
print("Shape of the events dataset:",df_events.shape)
print("Shape of the items properties part 1 dataset:", df_items1.shape)
print("Shape of the items properties part 2 dataset:", df_items2.shape)

Shape of category dataset: (1669, 2)
Shape of the events dataset: (2756101, 5)
Shape of the items properties part 1 dataset: (10999999, 4)
Shape of the items properties part 2 dataset: (9275903, 4)


In [ ]:
df_sorted_events=pd.read_pickle("/content/drive/MyDrive/AIPI Assignment-3/SA2C_code/SA2C_code/Kaggle/data/sorted_events.df")

In [ ]:
df_items = pd.concat([df_items1, df_items2], axis=0)
df_items.shape

(20275902, 4)

In [ ]:

events_item = df_sorted_events.item_id.unique()

In [ ]:

event_item_features_df = df_items[(df_items.itemid.isin(events_item)) &
                                   (df_items['property'] == 'categoryid')].drop(['property', 'timestamp'], axis=1).drop_duplicates()


missing_items_df = pd.DataFrame(set(events_item) - set(event_item_features_df.itemid.unique()), columns=['itemid'])
missing_items_df['value'] = np.nan


event_item_features_df = pd.concat([event_item_features_df, missing_items_df], axis=0)
event_item_features_df.columns = ['itemid', 'categoryid']
event_item_features_df.categoryid = event_item_features_df.categoryid.astype(float)

event_category_features_df = event_item_features_df.merge(df_category, on='categoryid', how='left').drop_duplicates()
parent_category_ids = event_category_features_df.parentid.unique()


In [ ]:
parent_category_ids.shape

(282,)

In [ ]:
data = []
for item in events_item:
  row = event_category_features_df[event_category_features_df.itemid == item]
  if len(row) == 0:
    categorical = parent = np.nan
  else:
    categorical = row.categoryid.tolist()
    parent = row.parentid.tolist()

  data.append({'itemid': item, 'categoryid': categorical, 'parentid':parent})

item_features_ = pd.DataFrame(data)

In [ ]:
one_hot_encoding = []
for idx in range(item_features_.shape[0]):
  one_hot_encoding.append(np.isin(parent_category_ids,
                                 item_features_.parentid[idx]).astype(int))

one_hot_encoding = np.array(one_hot_encoding)
one_hot_encoding = pd.DataFrame(one_hot_encoding, index = item_features_.itemid,
             columns = parent_category_ids)

In [ ]:
one_hot_encoding.shape

(70852, 282)

In [ ]:
one_hot_encoding.to_csv('/content/drive/MyDrive/AIPI Assignment-3/SA2C_code/SA2C_code/Kaggle/data/item_features.csv')

In [ ]:
!tf_upgrade_v2 \
  --infile 'SA2C.py' \
  --outfile 'SA2C_new.py' \
  --reportfile report_SA2C.txt

2023-11-24 18:48:24.479376: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 18:48:24.479431: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 18:48:24.479470: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 18:48:24.487221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 18:48:25.769254: W tensorflow/c

In [ ]:
! pip install pandas trfl

In [ ]:
!tf_upgrade_v2 \
  --infile 'SNQN_Features.py' \
  --outfile 'SNQN_Features_new.py' \
  --reportfile report_SA2C.txt

2023-11-24 18:52:18.993646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 18:52:18.993704: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 18:52:18.993745: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 18:52:19.001357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 18:52:20.219156: W tensorflow/c

In [ ]:
!tf_upgrade_v2 \
  --infile 'SNQN.py' \
  --outfile 'SNQN_new.py' \
  --reportfile report_SNQN.txt

2023-11-24 18:57:16.471078: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 18:57:16.471129: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 18:57:16.471170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 18:57:16.478622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 18:57:17.554509: W tensorflow/c

In [ ]:
! python SA2C_new.py --model=GRU --epoch=5

2023-11-15 21:37:02.315218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 21:37:02.315268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 21:37:02.315307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 21:37:02.322789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 21:37:03.437196: W tensorflow/c

In [ ]:
! python SA2C_new.py --model=Caser --epoch=5

2023-11-16 23:37:46.904971: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 23:37:46.905042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 23:37:46.905083: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 23:37:46.912849: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 23:37:48.049610: W tensorflow/compiler/

In [ ]:
! python SNQN_new.py --model=GRU --epoch=5

2023-11-24 19:11:56.820195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 19:11:56.820256: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 19:11:56.820295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 19:11:56.831476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:11:58.505952: W tensorflow/c

In [ ]:
!tf_upgrade_v2 \
  --infile 'SNQN_Features.py' \
  --outfile 'SNQN_Features_new.py' \
  --reportfile report_SA2C.txt

2023-11-25 01:27:13.358756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 01:27:13.358827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 01:27:13.358872: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 01:27:13.371971: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 01:27:14.789960: W tensorflow/c

In [ ]:
! python SNQN_Features_new.py --model=GRU --epoch=5

2023-11-25 02:28:56.510605: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 02:28:56.510656: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 02:28:56.510694: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 02:28:56.518709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 02:28:57.828588: W tensorflow/c

## Observations

The above results show the key metrics at different top k recommendations (5, 10, 15 and 20). SNQN_Features_new.py have the item features included. The inclusion of item features has improved the performance this is seen when we compare the key metrics throughout the training process. In both the models that is SNQN_new.py (does not contain item features) and SNQN_Features_new.py(contains item features) we can see that the metrics improve as the number of top-k recommendations increase, we can also see that for both the models the loss reduces, since the loss is reducing we can say that the models are learning. Since the model with item features is performing better we can say that including item features is beneficial in improving the recommendation quality.

## Conclusion

Successfully trained and compared session base DRL recommenders with and without item features. The inclusion of item features will help in improving the recommendation performance for e commerce applications
